In [1]:
from pyspark.sql import SparkSession, Row as s_Row
from pyspark.sql.functions import col as s_col, round as s_round, lit as s_lit
from pyspark.sql.types import StringType

import pandas as pd

import gc

#### Creating spark context

In [2]:
spark_context = (SparkSession.
        builder.
        appName("RatingHistogram").
        master("local[*]").
        getOrCreate())

spark_context

23/08/31 00:35:00 WARN Utils: Your hostname, IdeaPad-L340 resolves to a loopback address: 127.0.1.1; using 192.168.100.16 instead (on interface wlp1s0)
23/08/31 00:35:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/31 00:35:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/31 00:35:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


#### Manipulating frames with pyspark

##### Reading file and showing content

In [3]:
file_path = "data/ml-100k/u.data"
ratings_sdf = spark_context.read.csv(path=file_path, sep="\t", inferSchema=True)
ratings_sdf.show(5)

+---+---+---+---------+
|_c0|_c1|_c2|      _c3|
+---+---+---+---------+
|196|242|  3|881250949|
|186|302|  3|891717742|
| 22|377|  1|878887116|
|244| 51|  2|880606923|
|166|346|  1|886397596|
+---+---+---+---------+
only showing top 5 rows



In [4]:
ratings_sdf.describe().show()

23/08/31 00:35:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+-----------------+
|summary|               _c0|               _c1|               _c2|              _c3|
+-------+------------------+------------------+------------------+-----------------+
|  count|            100000|            100000|            100000|           100000|
|   mean|         462.48475|         425.53013|           3.52986|8.8352885148862E8|
| stddev|266.61442012750905|330.79835632558473|1.1256735991443214|5343856.189502848|
|    min|                 1|                 1|                 1|        874724710|
|    max|               943|              1682|                 5|        893286638|
+-------+------------------+------------------+------------------+-----------------+



##### Renaming columns and showing schema

In [5]:
ratings_sdf = ratings_sdf.select(
                    s_col("_c0").alias("unknown_1"), 
                    s_col("_c1").alias("unknown_2"),
                    s_col("_c2").alias("ratings"),
                    s_col("_c3").alias("unknown_4")
                )

In [6]:
ratings_sdf.printSchema()

root
 |-- unknown_1: integer (nullable = true)
 |-- unknown_2: integer (nullable = true)
 |-- ratings: integer (nullable = true)
 |-- unknown_4: integer (nullable = true)



##### Operations over spark dataframes

In [7]:
ratings_stars = ratings_sdf.select("ratings")
ratings_stars_counted = ratings_stars.groupBy("ratings").count()
ratings_stars_counted.show()

+-------+-----+
|ratings|count|
+-------+-----+
|      1| 6110|
|      3|27145|
|      5|21201|
|      4|34174|
|      2|11370|
+-------+-----+



In [8]:
percentage_ratings = 100 * (ratings_stars_counted["count"] / 100_000)
percentage_ratings = s_round(percentage_ratings, 2)
ratings_stars_counted = ratings_stars_counted.withColumn("percentage_ratings", percentage_ratings)

ratings_stars_counted = ratings_stars_counted.orderBy("ratings")

ratings_stars_counted.show()

+-------+-----+------------------+
|ratings|count|percentage_ratings|
+-------+-----+------------------+
|      1| 6110|              6.11|
|      2|11370|             11.37|
|      3|27145|             27.15|
|      4|34174|             34.17|
|      5|21201|              21.2|
+-------+-----+------------------+



##### Droping and adding (columns / rows)

In [9]:
# Drop to remove columns
ratings_stars_percentage = ratings_stars_counted.drop("count")
ratings_stars_percentage.show()

+-------+------------------+
|ratings|percentage_ratings|
+-------+------------------+
|      1|              6.11|
|      2|             11.37|
|      3|             27.15|
|      4|             34.17|
|      5|              21.2|
+-------+------------------+



In [10]:
# WithColumnRenamed to add a new column
ratings_stars_percentage = ratings_stars_percentage.withColumnRenamed("ratings", "stars")
ratings_stars_percentage

DataFrame[stars: int, percentage_ratings: double]

In [11]:
# Union to concat or add new rows
new_rows = [[6, 0],
            [7, 0]]
columns = ratings_stars_percentage.columns

new_row = spark_context.createDataFrame(new_rows, columns)

ratings_stars_percentage = ratings_stars_percentage.union(new_row)
ratings_stars_percentage.show()

+-----+------------------+
|stars|percentage_ratings|
+-----+------------------+
|    1|              6.11|
|    2|             11.37|
|    3|             27.15|
|    4|             34.17|
|    5|              21.2|
|    6|               0.0|
|    7|               0.0|
+-----+------------------+



In [12]:
# Filter or Where to remove rows
ratings_stars_percentage = ratings_stars_percentage.filter(ratings_stars_percentage["stars"] < 6)
ratings_stars_percentage.show()

+-----+------------------+
|stars|percentage_ratings|
+-----+------------------+
|    1|              6.11|
|    2|             11.37|
|    3|             27.15|
|    4|             34.17|
|    5|              21.2|
+-----+------------------+



In [13]:
# Filter by using another sintax
ratings_stars_percentage = ratings_stars_percentage.filter("stars < 6")
ratings_stars_percentage.show()

+-----+------------------+
|stars|percentage_ratings|
+-----+------------------+
|    1|              6.11|
|    2|             11.37|
|    3|             27.15|
|    4|             34.17|
|    5|              21.2|
+-----+------------------+



In [14]:
# Filter using different operations
mask_gt = (ratings_stars_percentage["percentage_ratings"] > 10)
mask_lt = (ratings_stars_percentage["percentage_ratings"] < 40)

ratings_stars_percentage.filter(mask_gt & mask_lt).show()

+-----+------------------+
|stars|percentage_ratings|
+-----+------------------+
|    2|             11.37|
|    3|             27.15|
|    4|             34.17|
|    5|              21.2|
+-----+------------------+



##### Null values

In [15]:
# Adding an empty colum
ratings_stars_percentage = ratings_stars_percentage.withColumn(
                                "Null_column",
                                s_lit(None).cast(StringType())
)

ratings_stars_percentage.show()

+-----+------------------+-----------+
|stars|percentage_ratings|Null_column|
+-----+------------------+-----------+
|    1|              6.11|       null|
|    2|             11.37|       null|
|    3|             27.15|       null|
|    4|             34.17|       null|
|    5|              21.2|       null|
+-----+------------------+-----------+



In [16]:
# Creating empty rows
new_row_1 = s_Row(stars=0, percentage_ratings=None, Null_column=None)
new_row_2 = s_Row(stars=None, percentage_ratings=0, Null_column=0)
new_rows = [new_row_1, new_row_2]

# Merging frames
semi_empty_sdf = spark_context.createDataFrame(new_rows, ["stars","percentage_ratings","Null_column"])
ratings_stars_percentage = ratings_stars_percentage.union(semi_empty_sdf)
ratings_stars_percentage.show()

+-----+------------------+-----------+
|stars|percentage_ratings|Null_column|
+-----+------------------+-----------+
|    1|              6.11|       null|
|    2|             11.37|       null|
|    3|             27.15|       null|
|    4|             34.17|       null|
|    5|              21.2|       null|
|    0|              null|       null|
| null|               0.0|          0|
+-----+------------------+-----------+



In [17]:
# Drop columns and rows with values
ratings_stars_percentage.na.drop().show()

+-----+------------------+-----------+
|stars|percentage_ratings|Null_column|
+-----+------------------+-----------+
+-----+------------------+-----------+



In [18]:
# Drop using all or any (default any) and threshhold as a 
# metric of tolerance againts null values
ratings_stars_percentage.na.drop(how="any", thresh=2).show()

+-----+------------------+-----------+
|stars|percentage_ratings|Null_column|
+-----+------------------+-----------+
|    1|              6.11|       null|
|    2|             11.37|       null|
|    3|             27.15|       null|
|    4|             34.17|       null|
|    5|              21.2|       null|
| null|               0.0|          0|
+-----+------------------+-----------+



In [19]:
# Drop taking into account only specific columns and those row that are no NULL
ratings_stars_percentage.na.drop(how="any", subset=["stars"]).show()

+-----+------------------+-----------+
|stars|percentage_ratings|Null_column|
+-----+------------------+-----------+
|    1|              6.11|       null|
|    2|             11.37|       null|
|    3|             27.15|       null|
|    4|             34.17|       null|
|    5|              21.2|       null|
|    0|              null|       null|
+-----+------------------+-----------+



In [20]:
# filling null values across the dataframe
ratings_stars_percentage.na.fill("empty_value").show()

+-----+------------------+-----------+
|stars|percentage_ratings|Null_column|
+-----+------------------+-----------+
|    1|              6.11|empty_value|
|    2|             11.37|empty_value|
|    3|             27.15|empty_value|
|    4|             34.17|empty_value|
|    5|              21.2|empty_value|
|    0|              null|empty_value|
| null|               0.0|          0|
+-----+------------------+-----------+



In [21]:
# filling null values only in specific columns. It'll only work if the datatype
# is the correct
ratings_stars_percentage.na.fill({"stars":"empty_value", "percentage_ratings":"empty_value"}).show()

+-----+------------------+-----------+
|stars|percentage_ratings|Null_column|
+-----+------------------+-----------+
|    1|              6.11|       null|
|    2|             11.37|       null|
|    3|             27.15|       null|
|    4|             34.17|       null|
|    5|              21.2|       null|
|    0|              null|       null|
| null|               0.0|          0|
+-----+------------------+-----------+



##### GroupBy

In [22]:
# Loading toy dataset
superstore_df = pd.read_excel("data/superstore.xls")
superstore_df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [23]:
# Transforming into a pyspark dataframe
superstore_sdf = spark_context.createDataFrame(superstore_df)
superstore_sdf.describe().show()

/home/lromero/mambaforge/envs/DataScience/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:479: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


+-------+------------------+--------------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+-------------------+------------------+-------------------+------------------+
|summary|            Row ID|      Order ID|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|    City|  State|       Postal Code| Region|     Product ID|  Category|Sub-Category|        Product Name|              Sales|          Quantity|           Discount|            Profit|
+-------+------------------+--------------+--------------+-----------+------------------+-----------+-------------+--------+-------+------------------+-------+---------------+----------+------------+--------------------+-------------------+------------------+-------------------+------------------+
|  count|              9994|          9994|          9994|       9994|              9994|       9994|  

In [24]:
# Using directly a method to get the most profitable customers
most_profitable_customers_sdf = (
                                    superstore_sdf.select("Customer Name", "Profit").
                                    groupBy("Customer Name").
                                    sum()
                                )

most_profitable_customers_sdf = most_profitable_customers_sdf.select(
                                        "Customer Name",
                                        s_round("sum(Profit)", 2).alias("total_profit")
                                    )

most_profitable_customers_sdf
most_profitable_customers_sdf.orderBy("total_profit", ascending = False).show(10)

+--------------------+------------+
|       Customer Name|total_profit|
+--------------------+------------+
|        Tamara Chand|     8981.32|
|        Raymond Buch|      6976.1|
|        Sanjit Chand|     5757.41|
|        Hunter Lopez|     5622.43|
|       Adrian Barton|     5444.81|
|        Tom Ashbrook|     4703.79|
|Christopher Martinez|     3899.89|
|       Keith Dawkins|     3038.63|
|         Andy Reiter|     2884.62|
|       Daniel Raglin|     2869.08|
+--------------------+------------+
only showing top 10 rows



In [25]:
# Using agg to get the most active customers (number of products bought)
most_active_customers_sdf = (
                                superstore_sdf.select("Customer Name", "Quantity").
                                groupBy("Customer Name").
                                agg({"Quantity":"sum"})
                            )

most_active_customers_sdf = most_active_customers_sdf.select(
                                        "Customer Name",
                                        s_col("sum(Quantity)").alias("total_products_brought")
                                    )

most_active_customers_sdf
most_active_customers_sdf.orderBy("total_products_brought", ascending = False).show(10)


+-------------------+----------------------+
|      Customer Name|total_products_brought|
+-------------------+----------------------+
|   Jonathan Doherty|                   150|
|      William Brown|                   146|
|           John Lee|                   143|
|         Paul Prost|                   138|
|  Steven Cartwright|                   133|
|         Emily Phan|                   124|
|Chloris Kastensmidt|                   122|
|  Cassandra Brandow|                   122|
|       Edward Hooks|                   120|
|       Matt Abelman|                   117|
+-------------------+----------------------+
only showing top 10 rows



In [27]:
spark_context.stop()